In [ ]:
import pandas as pd
import numpy as np
from cdcqr.common.config import LOCAL_DATA_DIR
from cdcqr.common.utils import load_df, save, resample_pv
from cdcqr.common.stats_utils import remove_outlier
import sys
sys.path.append('/github/cryptoderiv-quant/')
import matplotlib.pyplot as plt
%matplotlib inline
from cdcqr.backtest.utils import pos2pnl

from cdcqr.backtest.utils import signal_backtester_light

In [ ]:
from ct.alfafactory import runsignallight
from ct.fs import addma

In [ ]:
a1 = 'BTCUSD_PERP@binance'

In [ ]:
df_btc = load_df(a1)
df_btc.head()

In [ ]:
res=[]
for freq in ['1Min', '3Min', '5Min', '10Min', '30Min']:
    price = df_btc['c'].resample(freq).last().to_frame()
    for lbw in [5,10,20,30,60,120]:
        price['ma'] = price['c'].rolling(lbw).mean()
        for nstd in [2,3,4]:
            price['std'] = price['c'].rolling(lbw).std()
            price['hi'] = price['ma']+nstd*price['std']
            price['lo'] = price['ma']-nstd*price['std']
            for short in [True, False]:
                if short:
                    signal = price['c']>price['hi']
                else:
                    signal = price['c']<price['lo']
                num_signal = signal.sum()
                
                for n in [100,200,500,1000]:
                    ressig=runsignallight(price,signal=signal,n=n,short=short, fees=0.0007)
                    resd={"freq":freq, "lbw":lbw, 'short':short,'nstd':nstd,'n':n,"p":ressig, 'num_signal':num_signal}        
                    res.append(resd)

In [ ]:
pd.DataFrame(res).query('num_signal>50').groupby(['nstd','freq','lbw'])['p'].mean()

In [ ]:
pd.DataFrame(res).query('num_signal>50').pv()

In [ ]:
pd.DataFrame(res).query('num_signal>50').pv()

### using new backtester

In [ ]:
res=[]
for freq in ['1Min', '3Min', '5Min', '10Min', '30Min', '1H']:
    print(freq)
    df = df_btc['c'].resample(freq).last().to_frame()
    for lbw in [10,20,30,60,120,240]:
        print(lbw)
        df['ma'] = df['c'].rolling(lbw).mean()
        for nstd in [2.5,3, 3.5, 4, 4.5]:
            df['std'] = df['c'].rolling(lbw).std()
            df['hi'] = df['ma']+nstd*df['std']
            df['lo'] = df['ma']-nstd*df['std']
            df['signal'] = 0 
            df['signal'][df['c']>df['hi']]=-1
            df['signal'][df['c']<df['lo']]= 1
            for n in [100,200,500,1000]:
                resd = signal_backtester_light(df['c'], df['signal'], n=n)
                resd['freq']=freq
                resd['lbw']=lbw
                resd['nstd']=nstd
                resd['n']=n
                res.append(resd)

In [ ]:
pd.DataFrame(res).query('cost2alpha>0 & avg_num_trades_per_day>5').sort_values(['cost2alpha', 'avg_num_trades_per_day'])

In [ ]:
pd.DataFrame(res).pv()

In [ ]:
df_btc_train = df_btc[df_btc.index<'20210901']
df_btc_test = df_btc[df_btc.index>='20210901']

In [ ]:
df_btc_train.shape,df_btc_test.shape

In [ ]:
res=[]
for freq in ['1Min']:
    df = df_btc_train['c'].resample(freq).last().to_frame()
    for lbw in [10, 20, 30, 60, 120, 240]:
        print(lbw)
        df['ma'] = df['c'].rolling(lbw).mean()
        for nstd in [2, 2.5, 3, 3.5, 4, 4.5, 5]:
            df['std'] = df['c'].rolling(lbw).std()
            df['hi'] = df['ma']+nstd*df['std']
            df['lo'] = df['ma']-nstd*df['std']
            df['signal'] = 0 
            df['signal'][df['c']>df['hi']]=-1
            df['signal'][df['c']<df['lo']]= 1
            for n in [10, 20, 50, 100,200,500,1000]:
                resd = signal_backtester_light(df['c'], df['signal'], n=n)
                resd['lbw']=lbw
                resd['nstd']=nstd
                resd['n']=n
                resd['freq']='1Min'
                res.append(resd)

In [ ]:
cond = 'cost2alpha>0 & avg_num_trades_per_day>2 & cost2alpha<0.9'
pd.DataFrame(res).query(cond).sort_values(['cost2alpha', 'avg_num_trades_per_day'])

### signal combination

In [ ]:
def bb_params2pos(df0, params_dict,lag=1):
    freq=params_dict['freq'].values[0]
    lbw=params_dict['lbw'].values[0]
    nstd=params_dict['nstd'].values[0]
    n = params_dict['n'].values[0]
    
    df = df0['c'].resample(freq).last().to_frame()
    df['ma'] = df['c'].rolling(lbw).mean()
    df['std'] = df['c'].rolling(lbw).std()
    df['hi'] = df['ma']+nstd*df['std']
    df['lo'] = df['ma']-nstd*df['std']
    df['signal'] = 0 
    df['signal'][df['c']>df['hi']]=-1
    df['signal'][df['c']<df['lo']]= 1
    df['share_pos'] = df['signal'].shift(lag).replace(0, np.nan).ffill(limit=n).fillna(0)
    df['share_pos'][-1] = 0
    return df['share_pos']

In [ ]:
def bb_params2io(df0, params_dict,lag=1):
    freq=params_dict['freq'].values[0]
    lbw=params_dict['lbw'].values[0]
    nstd=params_dict['nstd'].values[0]
    n = params_dict['n'].values[0]

    df = df0['c'].resample(freq).last().to_frame()
    df['ma'] = df['c'].rolling(lbw).mean()
    df['std'] = df['c'].rolling(lbw).std()
    df['hi'] = df['ma']+nstd*df['std']
    df['lo'] = df['ma']-nstd*df['std']
    df['signal'] = 0 
    df['signal'][df['c']>df['hi']]=-1
    df['signal'][df['c']<df['lo']]= 1
    df['share_pos'] = df['signal'].shift(lag).replace(0, np.nan).ffill(limit=n).fillna(0)
    df['share_pos'][-1] = 0
    ret = pos2pnl(df)
    return ret['io']

In [ ]:
selected_params = pd.DataFrame(res).query(cond)
selected_params[['freq','lbw','nstd','n']].iloc[0,:].to_dict()

In [ ]:
pos1 = bb_params2pos(df_btc_test, selected_params[['freq','lbw','nstd','n']].iloc[0,:].to_dict())

In [ ]:
df_params = selected_params[['freq','lbw','nstd','n']]

In [ ]:
selected_params

### out of sample test

In [ ]:
df11 = df_params.groupby(['freq','lbw','nstd','n']).apply(lambda x : bb_params2pos(df_btc_train, x)).T

In [ ]:
df11.head()

In [ ]:
df11 = df_params.groupby(['freq','lbw','nstd','n']).apply(lambda x : bb_params2io(df_btc_test, x)).T

In [ ]:
df11